In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
from pprint import pprint
import json
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import re

import dill
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import mplcyberpunk
import seaborn as sns
import scipy as sp
import scipy.stats as stats
from typing import List, Dict, Callable

# plt.style.use('ggplot')
plt.style.use("cyberpunk")

import os
os.getcwd()

'/home/jovyan/work/Capstone_3/Skincare-Recommender'

In [3]:
# Load Time Consuming Scraping Session Variables to Translate to DataFrames  
dill.load_session('dill/notebook_env.db')

# 1: Drunk Elephant - Protini Polypeptide Cream
url_1, items_1, users_1 = item_1, item_info_1, user_info_1

# 2: TATCHA - The Water Cream
url_2, items_2, users_2 = item_2, item_info_2, user_info_2

# 3: BELIF - The True Cream Aqua Bomb
url_3, items_3, users_3 = item_3, item_info_3, user_info_3

# 4: BOSCIA - Cactus Water Moisturizer
url_4, items_4, users_4 = item_4, item_info_4, user_info_4

# 5: OLEHENRIKSEN - C-Rush™ Vita
url_5, items_5, users_5 = item_5, item_info_5, user_info_5

# 6: CLINIQUE - Dramatically Different Moisturizing Gel
url_6, items_6, users_6 = item_6, item_info_6, user_info_6

# 7: ORIGINS - Ginzing Energy Boosting Gel Moisturizer
url_7, items_7, users_7 = item_7, item_info_7, user_info_7

# 8: JOSIE MARAN - 100 Percent Pure Argan Oil
url_8, items_8, users_8 = item_8, item_info_8, user_info_8

# 9: FIRST AID BEAUTY - Ultra Repair® Cream Intense Hydration
url_9, items_9, users_9 = item_9, item_info_9, user_info_9

# 10: Dr. Jart - Cicapair™ Tiger Grass Color Correcting Treatment
url_10, items_10, users_10 = item_10, item_info_10, user_info_10

item_urls = [url_1, url_2, url_3, url_4, url_5, url_6, url_7, url_8, url_9, url_10]
item_collection = [items_1, items_2, items_3, items_4, items_5, items_6, items_7, items_8, items_9, items_10]
user_collection = [users_1, users_2, users_3, users_4, users_5, users_6, users_7, users_8, users_9, users_10]

## Create Item DataFrame

In [4]:
# Item Structure: item_url, item_brand, item_name, n_loves, avg_stars, ingredients, skin_target, n_reviews, five_stars, four_stars, three_stars, two_stars, one_stars, review_corpus

In [5]:
# Create item dataframe
item_df = pd.DataFrame({
    'item_url':item_urls,
    'item_brand':[x[0] for x in item_collection],
    'item_name':[x[1] for x in item_collection],
    'n_loves':[x[2] for x in item_collection],
    'avg_stars':[x[3] for x in item_collection], 
    'ingredients':[x[4] for x in item_collection], 
    'skin_target':[x[5] for x in item_collection], 
    'n_reviews':[x[6] for x in item_collection], 
    'star_dist':[x[7] for x in item_collection],
    'review_corpus':[x[8] for x in item_collection]
})

In [6]:
# Add Granulated star distribution
item_df['five_stars'] = item_df['star_dist'].apply(lambda x: int(x[0].partition(':')[2]))
item_df['four_stars'] = item_df['star_dist'].apply(lambda x: int(x[1].partition(':')[2]))
item_df['three_stars'] = item_df['star_dist'].apply(lambda x: int(x[2].partition(':')[2]))
item_df['two_stars'] = item_df['star_dist'].apply(lambda x: int(x[3].partition(':')[2]))
item_df['one_stars'] = item_df['star_dist'].apply(lambda x: int(x[4].partition(':')[2]))

item_df.drop(columns=['star_dist'], inplace=True)

In [7]:
item_df.head()

,item_url,item_brand,item_name,n_loves,avg_stars,ingredients,skin_target,n_reviews,review_corpus,five_stars,four_stars,three_stars,two_stars,one_stars
0,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,Protini™ Polypeptide Moisturizer,259473,4 stars,"Water/Aqua/Eau, Dicaprylyl Carbonate, Glycerin...","Normal, Dry, Combination, and Oily",4395 reviews,[11 h ago\nbeware of a breakout\nSoooo at firs...,2557,540,402,405,491
1,https://www.sephora.com/product/the-water-crea...,Tatcha,The Water Cream,233187,4 stars,"Water, Saccharomyces/Camellia Sinensis Leaf/Cl...",✔ Normal✔ Oily✔ Combination✔ Sensitive\r\n,3030 reviews,"[5 h ago\nGreat packaging, underwhelming produ...",1817,392,307,232,282
2,https://www.sephora.com/product/the-true-cream...,belif,The True Cream Aqua Bomb,191092,4.5 stars,"Water, Dipropylene Glycol, Glycerin, Methl Tri...","Normal, Dry, Combination, and Oily",4527 reviews,[5 h ago\nNot for oily/combo skin\nCompletely ...,3276,665,246,173,167
3,https://www.sephora.com/product/cactus-water-m...,boscia,Cactus Water Moisturizer,26641,4 stars,"Water, Butylene Glycol, Glycerin, Cyclopentasi...","Normal, Combination, and Oily\r",2221 reviews,[4 d ago\nSoftest Skin\nThe BEST moisturizer i...,1263,538,178,108,134
4,https://www.sephora.com/product/dramatically-d...,OLEHENRIKSEN,C-Rush™ Vitamin C Gel Moisturizer,80124,4.5 stars,Sources of vitamin C:,✔ Normal✔ Oily✔ Combination✔ Dry✔ Sensitive,2619 reviews,[1 d ago\nJust Ok\nIt has a great orangey smel...,1721,565,151,85,97


## Create User DataFrame

In [8]:
# User Structure: item_url, item_brand, item_name ,user_id, user_meta, rating, review
user_collection[0][0] # id, meta, rating, reivew, brand, item_name

['harleen253',
 'Eye Color Brown\nHair color Black\nSkin Tone Medium\nSkin Type Oily',
 '5 stars',
 '3 h ago\nThis moisturizer has worked really well for my skin! I have combo skin and it doesn’t make me greasy. It keeps my skin plump and moisturized. I’ve gone through three of these now and how no intention of using any other moisturizer!\nRecommends this product',
 'Drunk Elephant',
 'Protini™ Polypeptide Moisturizer']

In [9]:
# Create user dataframe
user_df = pd.DataFrame({
    'user_id':[x[0] for product in user_collection for x in product],
    'user_meta':[x[1] for product in user_collection for x in product], 
    'star_rating':[x[2] for product in user_collection for x in product], 
    'review':[x[3] for product in user_collection for x in product],
    'item_brand':[x[4] for product in user_collection for x in product],
    'item_name':[x[5] for product in user_collection for x in product],
})

In [10]:
# Granulate and Clean User Data

# Cast star rating to int
user_df['star_rating'] = user_df['star_rating'].apply(lambda x: re_max(x, int) if type(x) != int else x)

# Remap items to numbers
item_names = user_df['item_name'].unique().tolist()
name_map = dict(zip(item_names, [f'item_{i}' for i in range(1, len(item_names) + 1)]))
user_df['item_name'] = user_df['item_name'].map(name_map)

# Remap brands to numbers
item_brands = user_df['item_brand'].unique().tolist()
brand_map = dict(zip(item_brands, [f'brand_{i}' for i in range(1, len(item_brands) + 1)]))
user_df['item_brand'] = user_df['item_brand'].map(brand_map)


In [38]:
def parse_user_meta(txt:str) -> dict:
    res = {}
    meta = [x.lower() for x in txt.split('\n')]
    matches = ['age', 'eye color', 'hair color', 'skin tone', 'skin type']
    if type(txt) == dict:
        res = txt
    elif len(meta) <= 1:
        return res
    else:
        for meta_type in matches:
            tmp = 'None'
            for info in meta:
                if re_match_target(info, meta_type):
                    part = info.partition(meta_type)
                    tmp = part[-1]
                    break
            res[meta_type] = tmp
    return res

user_df['user_meta'] = user_df['user_meta'].apply(parse_user_meta)


# user_df['one_stars'] = user_df['star_dist'].apply(lambda x: int(x[4].partition(':')[2]))

# user_df.drop(columns=['user_meta'], inplace=True)

In [39]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      20000 non-null  object
 1   user_meta    20000 non-null  object
 2   star_rating  20000 non-null  int64 
 3   review       20000 non-null  object
 4   item_brand   20000 non-null  object
 5   item_name    20000 non-null  object
dtypes: int64(1), object(5)
memory usage: 937.6+ KB


In [40]:
user_df

,user_id,user_meta,star_rating,review,item_brand,item_name
0,harleen253,"{'age': 'None', 'eye color': ' brown', 'hair c...",5,3 h ago\nThis moisturizer has worked really we...,brand_1,item_1
1,supremekaylap15,"{'age': 'None', 'eye color': ' hazel', 'hair c...",2,14 h ago\nbeware of a breakout\nSoooo at first...,brand_1,item_1
2,Alybug97,"{'age': 'None', 'eye color': ' brown', 'hair c...",4,15 h ago\nlasts for a long time for the price!...,brand_1,item_1
3,Mikachew,{},2,15 h ago\nJust okay\nI got this as part of the...,brand_1,item_1
4,jkbean,"{'age': 'None', 'eye color': ' blue', 'hair co...",2,1 d ago\nBirthday gift was better.\nAlright. I...,brand_1,item_1
...,...,...,...,...,...,...
19995,Cristinaa18,"{'age': 'None', 'eye color': ' brown', 'hair c...",5,25 Dec 2017\nAmazing Product!\nThis product is...,brand_10,item_10
19996,octiskeet,"{'age': 'None', 'eye color': ' brown', 'hair c...",1,"23 Dec 2017\nTexture issues\nI really, really ...",brand_10,item_10
19997,genghisette,"{'age': 'None', 'eye color': 'None', 'hair col...",5,23 Dec 2017\nLove it.\nI accidentally bought t...,brand_10,item_10
19998,shiena42,"{'age': 'None', 'eye color': ' brown', 'hair c...",3,23 Dec 2017\nBeige means beige\nNot tan skin f...,brand_10,item_10
